# Creating a task with the Brewery model

This chapter is dedicated to tutorials on how to create a Task in the library. This step is essential as all the experiments in the library are performed on Tasks. A Task behaves as a function: it takes inputs, performs an evaluation and outputs the results. A Task defines a specific computation using a simulator. For example if we have a supply chain simulator one task may compute the profit given some input demand, and another task may compute the service level given some inventory policy input. The tasks are different but rely on the same simulator. The task therefore allows to specify which inputs we want to experiment with, to pre-process these inputs, to evaluate a simulator and to post-process the outputs of the simulation. 

Suppose we want to perform an experimentation on the Brewery model (e.g an optimization). For this we have to specify which KPI we want to optimize and with respect to which input parameters. A Task allows this specification in a way such that the experiment is able to run the task automatically. This tutorial explains how to define such a task.

In [1]:
# code to display the diagram below
from IPython.display import Image
Image(url= "Comets_Workflow.png", width=900)

## The Brewery model

We are working with a drink consumption model including stock level management and customer satisfaction dynamics. The model input parameters
that we will deal with are: 

- NBWaiters: the number of waiters in the bar
- RestockQty: a fix quantity of drinks added to the current stock of the bar once it reaches a threshold value.

The Brewery model has no default outputs as the value of all attributes at the end of a simulation can be considered as an output. In our case we will use the following attributes:

 - AverageSatisfaction: the average customer satisfaction
 - Stock: the stock of the bar.

## Importing the library

For this tutorial we only need to import comets:

In [2]:
import comets 

## Creating a task with a Cosmo model

Comets always works with the same method: **it performs analyses on Tasks**. A Task behaves as a function that takes as input a ParameterSet, perform an evaluation and outputs another ParameterSet. A ParameterSet consist of a set of parameters, stored as a dictionary containing names of parameters and their values. 

The Task has a method evaluate that is in charge of performing the evaluation and returning the outputs.
During the task evaluation, the simulation is run, but when creating the task we also allow for some additional pre-processing of the input parameters or post-processing of the outputs. Note that it is possible to create different Tasks with the same model by using different pre-processing and/or post-processing.

The task uses the Brewery model via a model interface and can additionally perform some encoding on the inputs and some posttreatment
on the outputs of the model. It allows the experimenter, for example, to perform an experiment on a KPI that is not computed within the model
but depends on the model outputs.

In order to create our Task we first we need to create a model interface that allows the library to manipulate the Brewery model. For that we use a CosmoInterface. It will be declared
with two arguments:

- simulator_path (mandatory): path to the simulation file inside the folder Simulation of the project.  
- project_path (optional): path to the project containing the model. This should be an absolute path. If not specified, assumes we are
  running from inside the project folder.  
  
  
Another possible argument is cold_input_parameter_set. It enables you to fix a parameter’s value at the beginning of the experiment. This parameter’s value will not change during the entire experiment. 

In [3]:
#This example assumes that this notebook is in the folder containing the Brewery project
#and that the model has been compiled with the python wrappers activated.

from pathlib import Path
cwd = Path().resolve()

simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
)

Now that the model interface is created we can declare a task that uses this CosmoInterface.
This is called a ModelTask, it will specify what is the CosmoInterface we are using, which encoding on the input parameters should be performed (optional) and what are the outcomes of interest of the task (optional).
Since the Brewery model has no default output parameters, we will specify at least the following arguments:

- modelinterface (mandatory): ModelInterface used by the task to communicate with a model.  
- get_outcomes (callable, optional): function taking as argument the modelinterface and returning a ParameterSet
  containing the outputs of the model.  



A get_outcomes function should take as input the CosmoInterface, and return a ParameterSet. It can use the methods of the CosmoInterface to access the model, in particular the get_outputs method allows to access the attributes of the model.

In [4]:
# code to display the diagram below
from IPython.display import Image
Image(url= "Comets_ModelTask.png", width=1100)

In [5]:
def get_outcomes(simulator):
    outputs = simulator.get_outputs(['Model::{Entity}MyBar::@AverageSatisfaction', 'Model::{Entity}MyBar::@Stock'])
    AverageSatisfaction = outputs['Model::{Entity}MyBar::@AverageSatisfaction']
    stock = outputs['Model::{Entity}MyBar::@Stock']
    return {'AverageSatisfaction' : AverageSatisfaction, 'Stock' : stock}

mytask = comets.ModelTask(modelinterface=simulator, get_outcomes=get_outcomes)

Note that we could also have used the get_consumers() method  instead of the get_outputs() to get the Stock's value. 

Now that the task is defined, the library will be able to execute it.
If we want to evaluate the task ourselves with specific inputs, we need to declare them in the format expected by the Cosmo model interface (since there is no encoding), which is a ParameterSet containing as keys the datapath to some attribute, and as value the value of the attribute at the beginning of the simulation:

$$
\text{parameter set} = \text{ {datapath1: value1, datapath2: value1} }
$$

In [6]:
parameter_set = {'Model::{Entity}MyBar::@NbWaiters': 4,
                 'Model::{Entity}MyBar::@RestockQty': 12}

output = mytask.evaluate(parameter_set)

output is a dictionary containing the results of the task evaluation:

In [7]:
print(output)

{'AverageSatisfaction': 1, 'Stock': 17}


### Encode function

Using an encoder allows to do transformations on the input parameters of the model task before they are passed on to the model. The input parameters of the
task no longer have to be in the format expected by the model as long as the encoder encodes the information correctly. Note that the input of the encoder should be in the ParameterSet's format.

For instance, instead of using the datapaths in the input ParameterSet like above, we can create an encoder that maps the names of the input parameters to their datapaths: 


In [8]:
def encoder(parameters):
    return {'Model::{Entity}MyBar::@NbWaiters': parameters['NbWaiters'],
            'Model::{Entity}MyBar::@RestockQty': parameters['RestockQty']}
    

Now we can create the model interface, the task and declare the encoder to use:

In [9]:
simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
)


mytask2 = comets.ModelTask(
    modelinterface = simulator,
    encode = encoder,
    get_outcomes = get_outcomes)

The task can now be evaluated with the new parameter set:

In [10]:
parameter_set = {'NbWaiters': 4,
                 'RestockQty': 12}

output2 = mytask2.evaluate(parameter_set)

output2 is a dictionary containing the results of the task evaluation:

In [11]:
print(output2)

{'AverageSatisfaction': 1, 'Stock': 17}


### get_outcomes function

In addition to selecting the model outputs, the get_outcomes function offers the possibility to compute other quantities that depend on
the model outputs, such as KPIs that are of interest to the experiment.

Imagine for example that the maximum stock capacity of the bar is 50 drinks. We want to return a percentage of this maximum capacity instead of the actual stock.  This quantity can be computed as follows: 

$$
\text{maximum capacity percentage} = \frac{\text{Stock}}{50}
$$

We are going to use a get_outcomes function to compute this quantity:

In [12]:
def get_outcomes2(simulator):
    output = simulator.get_outputs(['Model::{Entity}MyBar::@Stock'])
    stock = output['Model::{Entity}MyBar::@Stock']
    maximum_capacity_percentage = stock/50
    return { 'maximum capacity percentage' : maximum_capacity_percentage}

Now we can create the model interface, the task and declare the get_outcomes function to use:

In [13]:
simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
)


mytask3 = comets.ModelTask(
    modelinterface = simulator,
    encode = encoder,
    get_outcomes = get_outcomes2)

The task can now be evaluated:

In [14]:
parameter_set = {'NbWaiters': 4,
                 'RestockQty': 12}

output3 = mytask3.evaluate(parameter_set)

outputs is a dictionary containing the results of the task evaluation:

In [15]:
print(output3)

{'maximum capacity percentage': 0.3}


## Conclusion

You are now able to connect the Brewery model to the library via a task. The task plays a key role in all the experiments,
now that you know how to declare and use it, you can start experimenting!